In [29]:
from glob import glob
import os
import json

In [30]:
root_path = "../data/a9_preprocessed"

for split in ["train", "val", "test"]:
    img_label_s1_folder = os.path.join(
        root_path, split, "labels_images", "s110_camera_basler_south1_8mm"
    )
    img_label_s1_paths = sorted(glob(os.path.join(img_label_s1_folder, "*")))

    total_not_only_prev = 0
    total_not_only_next = 0
    total_not_both = 0

    data = {}

    for x in img_label_s1_paths:
        img_label_s1_json_path = os.path.join(x)
        json_data = None
        with open(img_label_s1_json_path, "r") as f:
            json_data = json.load(f)
            frame_idx = list(json_data["openlabel"]["frames"].keys())[0]
            frame_properties = json_data["openlabel"]["frames"][frame_idx]["frame_properties"]
            token = frame_properties["token"]
            prev = frame_properties["prev"]
            next = frame_properties["next"]
            scene_token = frame_properties["scene_token"]
            frame_idx = frame_properties["frame_idx"]
            if prev is None and next is None:
                total_not_both += 1
            elif prev is None:
                total_not_only_prev += 1
            elif next is None:
                total_not_only_next += 1

            data[token] = (prev, next, scene_token, frame_idx)

    chain_results = {}

    for token, (_, _, scene_token, _) in data.items():
        if scene_token not in chain_results:
            chain_results[scene_token] = {
                "total": 0,
                "frames": [token]
            }
        else:
            chain_results[scene_token]["frames"].append(token)

    for token, (prev, next, scene_token, frame_idx) in data.items(): 
        chain_results[scene_token]["total"] = len(chain_results[scene_token]["frames"])

    # def rec_chain_check(x, only_left: bool = False, only_right: bool = False) -> int:
    #     prev = data[x][0]
    #     next = data[x][1]
    #     data[x] = (prev, next, True)
    #     size = 1
    #     if not only_right and data[x][0] is not None:
    #         size = rec_chain_check(data[x][0], True, False) + 1
    #     if not only_left and data[x][1] is not None:
    #         size = rec_chain_check(data[x][1], False, True) + 1
    #     return size

    # chain_results = {}
    # for k in data.keys():
    #     if not data[k][2]:
    #         res = rec_chain_check(k)
    #         if str(res) not in chain_results:
    #             chain_results[str(res)] = 1
    #         else:
    #             chain_results[str(res)] += 1

    print("\n", "-" * 15, "split: ", split, "-" * 15)
    chain_results = sorted(chain_results.items(), key=lambda x:int(x[1]["total"]))

    print("No. Sequences with following number of frames: ")
    for x, y in chain_results:
        print("\tscene token", x, "\ttotal no frames: \t", y["total"])

    print("\nNumber of sequences: \t\t", len(chain_results))
    print("Samples with no prev: \t\t", total_not_only_prev)
    print("Samples with no next: \t\t", total_not_only_next)
    print("Samples with no prev nor next: \t", total_not_both)


 --------------- split:  train ---------------
No. Sequences with following number of frames: 
	scene token 4ac780f2ca564233ad571ff915f8a713 	total no frames: 	 238
	scene token a9adcc18f3024a59b44ab25846097a32 	total no frames: 	 242
	scene token ae822a87e1b3436abe55cc9acbe3ca15 	total no frames: 	 480
	scene token 589370aa155e48969bd8e1a9c0ecd912 	total no frames: 	 960

Number of sequences: 		 4
Samples with no prev: 		 4
Samples with no next: 		 4
Samples with no prev nor next: 	 0

 --------------- split:  val ---------------
No. Sequences with following number of frames: 
	scene token 864f2b15b5224955987f451315b9fb5d 	total no frames: 	 1
	scene token 146aff001267487cb70f1d8842e9d565 	total no frames: 	 1
	scene token 29e3f5b3c27340d7b849822607aaad54 	total no frames: 	 1
	scene token bdbee530cad240a0889fb73fb0480deb 	total no frames: 	 1
	scene token 43bf55c8553748ad8b7db2afaf70481a 	total no frames: 	 1
	scene token c80dc7a01d2b49a69e09cba26dc25a10 	total no frames: 	 2
	scene